In [20]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
# Load the .env file
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

In [21]:
llm = ChatOpenAI(model_name='gpt-4.1-nano',api_key=api_key)

In [22]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000025411D55350>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000025412146D50>, root_client=<openai.OpenAI object at 0x0000025411762750>, root_async_client=<openai.AsyncOpenAI object at 0x0000025411D55B10>, model_name='gpt-4.1-nano', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [1]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain_huggingface import HuggingFaceEmbeddings


client = QdrantClient(url="http://localhost:6333")
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en") 


c:\Users\KEVAL\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
collection_name = "Handbook_of_Clinical_Psychiatry"
collection_name2 ="Handbook_of_diagnostic_and_statistical_manual"

def VectosrStore(client,collection_name,embedding):
    return client,collection_name,embedding

vector_store_diag =VectosrStore(client,collection_name2,embeddings)
vector_store_drug = VectosrStore(client,collection_name,embeddings)
vector_store_treatment = VectosrStore(client,collection_name,embeddings)

In [13]:
def retrieve(query: str, vector_store):
    """Retrieve information related to a query."""
    retrieved_docs = vector_store[0].query_points(
                collection_name="Handbook_of_Clinical_Psychiatry",
                query=embeddings.embed_query(query),
                with_payload=True,
                limit=3
                ).points

    serialized = []
    for doc in range(len(retrieved_docs)):
        serialized.append(f"Source {doc+1}: \n{retrieved_docs[doc].payload.get('metadata')}\n" f"Content{doc+1}: \n{retrieved_docs[doc].payload.get('Content')}")
    return "\n\n\n".join(serialized), retrieved_docs

In [4]:
class DiagnosisAgent:
    def __init__(self, vector_store, llm):
        self.vector_store = vector_store
        self.llm = llm
        self.tools = [retrieve]
    def run(self, query):
        tool_messages = self.tools[0](query, self.vector_store)
        system_message_content = (
            "You are an Diagnosis assistant your tasks is to provide deep diagnosis result. "
            "Use the following pieces of retrieved context to answer "
            "the question. If you don't know the answer, say that you "
            "don't know. Give detailed answer"
            "\n\n"
            f"Question : {query}"
            "\n\n"
            "Context : \n"
            f"{tool_messages}"
        )
        prompt = system_message_content
        response = self.llm.invoke(prompt)
        return {"Response": [response]}

In [7]:
info= """
Patient_ID: 25/7891

  Age: 52
  Gender: Female

  Mental Status Examination :
    - Appearance -> Disheveled, Agitated
    - Speech:  -> Pressured and Slow
    - Mood and Affect -> Depressed and Anxious
    - Thought Process -> Disorganized and Tangential
    - Insight and Judgment: -> Partial


  Presenting Complaints (as described by the patient):
  1. Depressed Mood
  2. Anxiety/Restlessness

  History: Onset and Duration of Symptoms is 20 years

  Known psychiatric diagnoses: Alcohol Use disorder, bipolar disorder

  Family History: Mother with generalized anxiety disorder

  Allergies: 

  Current medications: Olanzapine 20,clozapibe 50,lithium 800,divaa 750,baclofen 30,clonazepam 0.5,Pan 40, Lorazepam 2mg

  Key Observations during Consultation:
    - Aggressive and abusive behavior
    - Big talks
    - Inflated self esteem
    - Decreased need for sleep
    - Suspicious over family members
    - Increased alcohol use


"""

In [9]:
print(info)


Patient_ID: 25/7891

  Age: 52
  Gender: Female

  Mental Status Examination :
    - Appearance -> Disheveled, Agitated
    - Speech:  -> Pressured and Slow
    - Mood and Affect -> Depressed and Anxious
    - Thought Process -> Disorganized and Tangential
    - Insight and Judgment: -> Partial


  Presenting Complaints (as described by the patient):
  1. Depressed Mood
  2. Anxiety/Restlessness

  History: Onset and Duration of Symptoms is 20 years

  Known psychiatric diagnoses: Alcohol Use disorder, bipolar disorder

  Family History: Mother with generalized anxiety disorder

  Allergies: 

  Current medications: Olanzapine 20,clozapibe 50,lithium 800,divaa 750,baclofen 30,clonazepam 0.5,Pan 40, Lorazepam 2mg

  Key Observations during Consultation:
    - Aggressive and abusive behavior
    - Big talks
    - Inflated self esteem
    - Decreased need for sleep
    - Suspicious over family members
    - Increased alcohol use





In [14]:
ret = retrieve(info,vector_store_diag)

In [37]:
ret[1]

[ScoredPoint(id=476, version=0, score=0.89596194, payload={'Content': 'Current mental status findings Hostile, irritable, menacing, threatening Agitation Victim(s) apparently picked out Weapons available Acute Intoxication Paranoia Delusions or hallucinations, especially command-type or that are used by patients to explain or justify their behavior Impaired empathy Disorders Mania (when characterized by prominent irritability), as in bipolar disorder or schizoaffective disorder, bipolar type Paranoid schizophrenia Anabolic steroid abuse Personality change (with disinhibition, e.g., frontal lobe syndrome) Dementia Delirium Mental retardation Paranoid personality disorder Antisocial personality disorder Borderline personality disorder Alcohol intoxication Stimulant intoxication (cocaine, amphetamines) Intermittent explosive disorder Delusional disorder Personal history History of violent behavior, impulsivity, in similar circumstances Recent act(s) of violence/destruction of property His

In [23]:
diag_agent = DiagnosisAgent(vector_store_diag, llm)

In [25]:
res = diag_agent.run(info)

In [30]:
print(res['Response'][0].content)

Based on the comprehensive clinical information provided for Patient ID 25/7891, the deep diagnostic impression is as follows:

**Primary Diagnosis:**
- **Bipolar I Disorder, current episode (Mixed) or possibly a Mixed Episode of Bipolar Disorder**

**Supporting findings for this diagnosis include:**
- **History of Bipolar Disorder:** Known psychiatric diagnosis with longstanding symptoms (20 years duration).
- **Current Clinical Presentation:**
  - Elevated psychomotor activity, increased talkativeness ("Big talks"), inflated self-esteem.
  - Decreased need for sleep.
  - Pressured but possibly slowed speech.
  - Mood appears to fluctuate between depression and agitation.
- **Behavioral Observations during consultation:**
  - Disorganized, tangential thought process.
  - Distrustful, suspicious over family members.
  - Aggressive, abusive behavior, and violent tendencies.
  - Inflated self-esteem suggests grandiosity.
  - Grandiosity aligns with manic/mixed features.
- **Additional Fe

In [41]:
context = ""
for i in ret[1]:
    context  = context + i.payload.get("Content") + "\n"

In [43]:
print(context)

Current mental status findings Hostile, irritable, menacing, threatening Agitation Victim(s) apparently picked out Weapons available Acute Intoxication Paranoia Delusions or hallucinations, especially command-type or that are used by patients to explain or justify their behavior Impaired empathy Disorders Mania (when characterized by prominent irritability), as in bipolar disorder or schizoaffective disorder, bipolar type Paranoid schizophrenia Anabolic steroid abuse Personality change (with disinhibition, e.g., frontal lobe syndrome) Dementia Delirium Mental retardation Paranoid personality disorder Antisocial personality disorder Borderline personality disorder Alcohol intoxication Stimulant intoxication (cocaine, amphetamines) Intermittent explosive disorder Delusional disorder Personal history History of violent behavior, impulsivity, in similar circumstances Recent act(s) of violence/destruction of property History of being physically abused in childhood Growing up in a family whe